## This notebook is for post-processing

In [ ]:
import os
import cv2
import numpy as np
import OpenLMlib as PSR

### Processing all frames

In [ ]:
main_folder = "path/to/main folder of all time lapse image folders"
for n in np.arange(0,263):
    subfolder = main_folder + str(n) + '/'
    scale_factor = 4
    file_name_list = list(np.arange(17, 22)) + list(np.arange(25, 30)) + \
                    list(np.arange(33, 38)) + list(np.arange(41, 46)) + \
                    list(np.arange(49, 54))
    frames = []
    for i in range(len(file_name_list)):
        file_path = subfolder + str(file_name_list[i]) + ".dng"
        npy_path = subfolder + str(file_name_list[i]) + ".npy"
        if os.path.exists(npy_path):
            os.remove(npy_path)
        img = PSR.largest_FOV(file_path)
        frames.append(img)
    frames = np.array(frames)
    frame_size = len(file_name_list)
    shift_map = np.load("path/to/shift_map.npy")
    HR_img = PSR.superResolution(frames, shift_map, frame_size, scale_factor)
    HR_path = subfolder + "HR_raw.npy"
    np.save(HR_path, HR_img)

    z_start = 9304*10**-6
    z_step = 1 * 10**-6
    iteration = 1
    reconstructed_imageHR, ToG, optimized_zs = PSR.imageReconstruction(HR_img,z_start,z_step,iteration)

    imgHR_path = subfolder + "HR_reconstructed.npy"
    np.save(imgHR_path, reconstructed_imageHR)
    Jimg_path = subfolder + "HR_reconstructed.jpg"
    jpg_img = PSR.npy2jpg(reconstructed_imageHR)
    jpg_img.save(Jimg_path)

### Write video

In [ ]:
main_folder = "path/to/main folder of all time lapse image folders"
output_path = main_folder + "growth_video.avi"
fps = 10  # Frames per second

# Create an empty list to store the images
images = []

# Loop through the range of subfolder indices (0 to 288)
for i in range(263):
    subfolder = os.path.join(main_folder, str(i))  # Subfolder path
    HR_path = os.path.join(subfolder, "HR_reconstructed.jpg")  # Full path to the image
    if os.path.exists(HR_path):  # Check if the image exists
        images.append(HR_path)

# Read the first image to get the dimensions (width, height)
frame = cv2.imread(images[0])
height, width, layers = frame.shape

# Set up the video writer (using MP4 codec here)
fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # 'mp4v' for .mp4
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (height, width))

# Loop through the images and write each one to the video
for image_path in images:
    img = cv2.imread(image_path)
    #img = img[1187:2187,3288:4288,:]
    video_writer.write(img)  # Write frame to video

# Release the video writer object
video_writer.release()

print("Video created successfully!")